# Индексы в pandas
При знакомстве с pandas мы видели два атрибута `index` и `columns`, которые возвращали соответственно индексы и колонки `DataFrame`. Эти два атрибута имеют тип `Index`. Это специальный тип в pandas, у которого есть множество полезных функций. 

`Index` сам по себе является отдельной структурой данных. К его элементам можно обращаться как к стандартным спискам в Python. `Index` поддерживает такие операции над множествами, как объединение, пересечение и симметрическая разность. Также при работе над двумя объектами типа `Series` или `DataFrame` их индексы выравниваются так, чтобы автоматически выполнять операции только над соответствующими элементами. 

Можно создать самостоятельный индекс из массива следующим образом

In [1]:
import pandas as pd
import numpy as np

In [2]:
index = pd.Index(np.arange(2, 10, 2))
index

Int64Index([2, 4, 6, 8], dtype='int64')

К элементам индекса можно обращаться как обычным спискам Python

In [3]:
index[::3]

Int64Index([2, 8], dtype='int64')

Индексу можно задать имя, которое будет отображаться при выводе `Series` и `DataFrame`

In [4]:
index.name = '#idx'
index

Int64Index([2, 4, 6, 8], dtype='int64', name='#idx')

In [5]:
pd.Series([1, 2, 3, 4], index=index)

#idx
2    1
4    2
6    3
8    4
dtype: int64

Индексы поддерживают стандартные операции над множествами с помощью операторов `&`, `|` и `^` для пересечения, объединения и симметрической разности соответственно. 

In [6]:
indA = pd.Index(np.arange(1, 8, 2))
indB = pd.Index(np.arange(3, 7))
print(indA)
print(indB)

Int64Index([1, 3, 5, 7], dtype='int64')
Int64Index([3, 4, 5, 6], dtype='int64')


In [7]:
indA & indB

Int64Index([3, 5], dtype='int64')

In [8]:
indA | indB

Int64Index([1, 3, 4, 5, 6, 7], dtype='int64')

In [9]:
indA ^ indB

Int64Index([1, 4, 6, 7], dtype='int64')

## Сохранение и выравнивание индексов
При выполнении операции над `Series` или `DataFrame` pandas сохраняет индексы и колонки исходных объектов

In [10]:
df = pd.DataFrame(np.arange(0, 9).reshape((3, 3)), 
                  columns=['A', 'B', 'C'], 
                  index=['first', 'second', 'third'])
df

,A,B,C
first,0,1,2
second,3,4,5
third,6,7,8


In [11]:
np.exp(df)

,A,B,C
first,1.000000,2.718282,7.389056
second,20.085537,54.598150,148.413159
third,403.428793,1096.633158,2980.957987


При выполнении бинарных операций индексы двух объектов выравниваются так, чтобы операция выполнялась над двумя соответствующими элементами. Создадим два объекта `Series` с одинаковыми индексами

In [12]:
countries = ['Afghanistan', 'Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan']
area = [652864, 2724900, 199951, 143100, 491210, 448978]
population = [34656032, 17987736, 6019480, 8734951, 5662544, 32979000]
gdp = [21, 156.189, 7.061, 27.802, 42.355, 68.324]
gini = [29, 26.4, 27.4, 30.8, 40.8, 36.7]

In [13]:
country_area = pd.Series(data=area[:-1], index=countries[:-1])
country_population = pd.Series(data=population[1:], index=countries[1:])
print(country_area)
print('--------------------------')
print(country_population)

Afghanistan      652864
Kazakhstan      2724900
Kyrgyzstan       199951
Tajikistan       143100
Turkmenistan     491210
dtype: int64
--------------------------
Kazakhstan      17987736
Kyrgyzstan       6019480
Tajikistan       8734951
Turkmenistan     5662544
Uzbekistan      32979000
dtype: int64


Если вычислить плотность населения из этих двух последовательностей, то pandas поделит территорию на количество населения соответствующей страны. При этом плотность населения для первой и последней страны будут пустыми, так как их нет одновременно в обоих последовательностях

In [14]:
country_area / country_population

Afghanistan          NaN
Kazakhstan      0.151487
Kyrgyzstan      0.033217
Tajikistan      0.016382
Turkmenistan    0.086747
Uzbekistan           NaN
dtype: float64

При работе с `DataFrame` выравниваются одновременно индексы и колонки

In [15]:
A = pd.DataFrame(np.arange(0, 4).reshape(2, 2),
                 columns=list('AB'),
                index=['first', 'second'])
A

,A,B
first,0,1
second,2,3


In [16]:
B = pd.DataFrame(np.arange(5, 14).reshape(3, 3),
                 columns=list('ABC'),
                index=['first', 'second', 'third'])
B

,A,B,C
first,5,6,7
second,8,9,10
third,11,12,13


In [17]:
A + B

,A,B,C
first,5.0,7.0,NaN
second,10.0,12.0,NaN
third,NaN,NaN,NaN


# Иерархические (множественные) индексы
До сих пор мы рассматривали только одномерные и двухмерные структуры данных. Однако часто данные имеют больше, чем два измерения. В большинстве случаев время является дополнительным измерением в данных. Например, если представить организационную структуру, то в качестве измерений можно взять департамент, должности внутри департамента и время, когда должности занимали в данном департаменте. Чтобы работать с многомерными данными в pandas используется иерархические индексы, также называемые множественными индексами.

Для знакомства с иерархическими индексами мы будем использовать данные Всемироного Банка. Их можно [скачать](http://databank.worldbank.org/data/reports.aspx?source=2&country=AFG,KAZ,KGZ,TJK,TKM,UZB) на сайте банка, где есть огромное количество фильтров, из которых можно составить нужную для себя информацию.

In [18]:
population = [33736494, 34656032, 
              17544126, 17797032, 
              5956900,  6082700, 
              8548651,  8734951, 
              5565284,  5662544, 
              31298900, 31848200]
area = [652860,  652860, 
        2724902, 2724902, 
        199949,  199949, 
        141376,  141376, 
        488100,  488100, 
        447400,  447400]
debt = [2487497000,   2403867000, 
        153381212000, 163757713000, 
        7510751000,   7876314000, 
        5001714000,   4876658000, 
        402877000,    508683000, 
        14854025000,  16282526000]
countries = ['Afghanistan', 'Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan']
year = [2015, 2016]

Как многие другие объекты, множественные индексы в pandas можно создать различными способами. Один из способов создать список кортежей (tuples) со всемозможными парами годов и стран. Для этого воспользуемся функцией `product` из модуля `itertools`, которая создает такие пары значений (декартовое произведение для элементов множества)

In [19]:
import itertools 
index = list(itertools.product(countries, year))
index

[('Afghanistan', 2015),
 ('Afghanistan', 2016),
 ('Kazakhstan', 2015),
 ('Kazakhstan', 2016),
 ('Kyrgyzstan', 2015),
 ('Kyrgyzstan', 2016),
 ('Tajikistan', 2015),
 ('Tajikistan', 2016),
 ('Turkmenistan', 2015),
 ('Turkmenistan', 2016),
 ('Uzbekistan', 2015),
 ('Uzbekistan', 2016)]

Из полученного списка можно создать множественный индекс с помощью объекта `pd.MultiIndex`

In [20]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['Afghanistan', 'Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan'], [2015, 2016]],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]])

Обратите внимание, что у индекса есть два атрибута: `levels` и `labels`. `levels` хранит значения, которые может принимать каждый из уровней индекса. `labels` хранит комбинацию значений каждого уровня, которое есть в наборе данных.

В `pd.MultiIndex` есть готовый метод `from_product`, который принимает список значений для каждого уровня и сам комбинирует каждое значение каждого уровня с помощью декартового произведения. Можно использовать этот метод напрямую вместо `itertools.product`

In [21]:
index = pd.MultiIndex.from_product([countries, year])
index.names = ['country', 'year']
index

MultiIndex(levels=[['Afghanistan', 'Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan'], [2015, 2016]],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['country', 'year'])

Полученный индекс можно передать для создания последовательности

In [22]:
pop_series = pd.Series(population, index=index)
pop_series

country       year
Afghanistan   2015    33736494
              2016    34656032
Kazakhstan    2015    17544126
              2016    17797032
Kyrgyzstan    2015     5956900
              2016     6082700
Tajikistan    2015     8548651
              2016     8734951
Turkmenistan  2015     5565284
              2016     5662544
Uzbekistan    2015    31298900
              2016    31848200
dtype: int64

Мы можем использовать множественный индекс для доступа к разным срезам данных. Например, можно вытащить все значения за 2015 год

In [23]:
pop_series.loc[:, 2015]

country
Afghanistan     33736494
Kazakhstan      17544126
Kyrgyzstan       5956900
Tajikistan       8548651
Turkmenistan     5565284
Uzbekistan      31298900
dtype: int64

Или значения по всем годам для одной страны

In [24]:
pop_series.loc['Uzbekistan', :]

country     year
Uzbekistan  2015    31298900
            2016    31848200
dtype: int64

С помощью метода `unstack` можно превратить последний вложенный уровень индекса в колонки и из одномерного `Series` получить двухмерных `DataFrame`

In [25]:
pop_df = pop_series.unstack()
pop_df

year,2015,2016
country,,
Afghanistan,33736494,34656032
Kazakhstan,17544126,17797032
Kyrgyzstan,5956900,6082700
Tajikistan,8548651,8734951
Turkmenistan,5565284,5662544
Uzbekistan,31298900,31848200


Метод `stack` является обратным методу `unstack` и превращает колонки в последний уровень множественного индекса

In [26]:
pop_df.stack()

country       year
Afghanistan   2015    33736494
              2016    34656032
Kazakhstan    2015    17544126
              2016    17797032
Kyrgyzstan    2015     5956900
              2016     6082700
Tajikistan    2015     8548651
              2016     8734951
Turkmenistan  2015     5565284
              2016     5662544
Uzbekistan    2015    31298900
              2016    31848200
dtype: int64

## Иерархические колонки
Точно так же как и индексы, колонки `DataFrame` могут быть многоуровневыми. Комбинирование иерархических колонок с индексами позволяет анализировать сложные структуры данных.

В этом примере рассмотрим различные этапы в процессе разработки ПО. Команда состоит из трех независимых групп: *Backend, Frontend, Mobile*. Каждая задача проходит четыре этапа от начала до завершения: *Backlog, Development, QA, Release*. Задача может относится к одному из типов: *Bug, Imporvement, Feature*. По срочности выполнения задача может быть *Urgent* или *Normal*. Значения в ячейках таблицы указывают количество задач. Мы будем использовать названия групп и типы задач в качестве уровней иерархических индексов, а этапы и срочность задач в качестве уровней иерархических колонок. В результате мы получим таблицу размером 9 × 8

In [28]:
teams = ['Backend', 'Frontend', 'Mobile']
task_types = ['Bug', 'Improvement', 'Feature']
stages = ['Backlog', 'Development', 'QA', 'Release']
urgency = ['Normal', 'Urgent']

index = pd.MultiIndex.from_product([teams, task_types],
                                   names=['teams', 'type'])
columns = pd.MultiIndex.from_product([stages, urgency],
                                     names=['stages', 'urgency'])
tasks = np.random.randint(1, 5, size=(9, 8))
tasks[np.random.rand(9, 8).argsort(0) > 1] = 0
process_board_df = pd.DataFrame(tasks, index=index, columns=columns)
process_board_df

stages               Backlog        Development            QA        Release  \
urgency               Normal Urgent      Normal Urgent Normal Urgent  Normal   
teams    type                                                                  
Backend  Bug               0      0           0      0      0      2       0   
         Improvement       4      2           0      0      0      0       0   
         Feature           0      3           4      0      0      0       0   
Frontend Bug               0      0           4      0      1      0       2   
         Improvement       0      0           0      0      0      0       0   
         Feature           0      0           0      0      0      4       0   
Mobile   Bug               3      0           0      0      0      0       0   
         Improvement       0      0           0      2      3      0       3   
         Feature           0      0           0      3      0      0       0   

stages                       
urgency              Urgent  
teams    type                
Backend  Bug              0  
         Improvement      0  
         Feature          3  
Frontend Bug              2  
         Improvement      0  
         Feature          0  
Mobile   Bug              0  
         Improvement      0  
         Feature          0

Мы можем посмотреть все задачи в Backlog следующим образом

In [29]:
process_board_df['Backlog']

urgency               Normal  Urgent
teams    type                       
Backend  Bug               0       0
         Improvement       4       2
         Feature           0       3
Frontend Bug               0       0
         Improvement       0       0
         Feature           0       0
Mobile   Bug               3       0
         Improvement       0       0
         Feature           0       0

Или только Urgent задачи в этапе Development. При этом мы получим `Series`, так как данные будут одномерными с иерархическим индексом

In [30]:
process_board_df['Development', 'Urgent']

teams     type       
Backend   Bug            0
          Improvement    0
          Feature        0
Frontend  Bug            0
          Improvement    0
          Feature        0
Mobile    Bug            0
          Improvement    2
          Feature        3
Name: (Development, Urgent), dtype: int32

Для выбора по индексам нужно использовать атрибут `loc`. Например, для выбора всех задач в Backend можно поступить следующим образом

In [31]:
process_board_df.loc['Backend']

stages      Backlog        Development            QA        Release       
urgency      Normal Urgent      Normal Urgent Normal Urgent  Normal Urgent
type                                                                      
Bug               0      0           0      0      0      2       0      0
Improvement       4      2           0      0      0      0       0      0
Feature           0      3           4      0      0      0       0      3

Для комбинированного выбора иерархических индексов с колонками необходимо использовать кортежи (`tuples`). Первый кортеж перечисляет значения для выбора на каждом уровне иерархического индекса, второй кортеж для выбора на каждом уровне иерархических колонок. Ниже приведен пример выбора задач в группе Backend с типом Bug и в этапах Development или QA. Так как мы оставили уровень для срочности задачи пустым, то выводятся задачи любой срочности

In [32]:
process_board_df.loc[('Backend','Bug'), (['Development', 'QA'], )]

stages       urgency
Development  Normal     0
             Urgent     0
QA           Normal     0
             Urgent     2
Name: (Backend, Bug), dtype: int32

Можно использовать срезы для выбора данных. Например, следующим образом можно вытащить задачи во всех этапах в группе Backend с типом Bug.

In [33]:
process_board_df.loc[('Backend','Bug'), :]

stages       urgency
Backlog      Normal     0
             Urgent     0
Development  Normal     0
             Urgent     0
QA           Normal     0
             Urgent     2
Release      Normal     0
             Urgent     0
Name: (Backend, Bug), dtype: int32

Пример выше вытаскивает все колонки. Однако этот синтаксис нельзя использовать для выбора по срезу только по определенному уровню. Например, если нужно вытащить все Normal задачи во всех этапах в Backend, то следующий код не будет работать, так как это синтаксическая ошибка

In [34]:
process_board_df.loc[('Backend',), (:, 'Normal')]

SyntaxError: invalid syntax (<ipython-input-34-111656d72b82>, line 1)

Чтобы этого добиться в pandas есть специальный объект `pd.IndexSlice`, который используется для получения срезов разных уровней иерархических индексов

In [35]:
idx = pd.IndexSlice
process_board_df.loc[('Backend',), (idx[:], 'Normal')]

stages,Backlog,Development,QA,Release
urgency,Normal,Normal,Normal,Normal
type,,,,
Bug,0,0,0,0
Improvement,4,0,0,0
Feature,0,4,0,0


Срезы в иерархических индексах работают только для отсортированных уровней. В нашем примере значения типов задач Bug, Imporvement, Feature не отсортированы в алфавитном порядке. Если попробовать использовать срезы для этого уровня, то мы получим ошибку

In [36]:
try:
    process_board_df.loc[('Mobile', idx[:]), :]
except Exception as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'MultiIndex Slicing requires the index to be fully lexsorted tuple len (2), lexsort depth (1)'


Чтобы добиться того, что мы хотим, необходимо отсортировать данные по индексу. Для этого есть метод `sort_index`

In [37]:
process_board_df.sort_index().loc[('Mobile', idx[:]), :]

stages             Backlog        Development            QA        Release  \
urgency             Normal Urgent      Normal Urgent Normal Urgent  Normal   
teams  type                                                                  
Mobile Bug               3      0           0      0      0      0       0   
       Feature           0      0           0      3      0      0       0   
       Improvement       0      0           0      2      3      0       3   

stages                     
urgency            Urgent  
teams  type                
Mobile Bug              0  
       Feature          0  
       Improvement      0

Комбинируя эти подходы можно строить очень сложные условия для выбора данных. В следующий примере выбирает задачи во всех группах с типом Improvement и Feature во всех этапах начиная с этапа Development со уровнем срочности

In [38]:
process_board_df.sort_index().loc[(idx[:], ['Improvement', 'Feature']), (idx['Development':], 'Normal')]

stages               Development     QA Release
urgency                   Normal Normal  Normal
teams    type                                  
Backend  Feature               4      0       0
         Improvement           0      0       0
Frontend Feature               0      0       0
         Improvement           0      0       0
Mobile   Feature               0      0       0
         Improvement           0      3       3

### Сброс индексов
Иногда необходимо сбросить индексы и получить обычный индекс по возрастанию, который pandas присваивает по умолчанию при создании объектов `Series` и `DataFrame`. Это может понадобиться, например, для того, чтобы перенумеровать индексы после фильтрации или чтобы создать иерархический индекс с новой структурой. 

Для сброса индекса можно использовать метод `reset_index`

In [39]:
pop_series

country       year
Afghanistan   2015    33736494
              2016    34656032
Kazakhstan    2015    17544126
              2016    17797032
Kyrgyzstan    2015     5956900
              2016     6082700
Tajikistan    2015     8548651
              2016     8734951
Turkmenistan  2015     5565284
              2016     5662544
Uzbekistan    2015    31298900
              2016    31848200
dtype: int64

In [40]:
pop_flat_df = pop_series.reset_index(name='pop')
pop_flat_df

,country,year,pop
0,Afghanistan,2015,33736494
1,Afghanistan,2016,34656032
2,Kazakhstan,2015,17544126
3,Kazakhstan,2016,17797032
4,Kyrgyzstan,2015,5956900
5,Kyrgyzstan,2016,6082700
6,Tajikistan,2015,8548651
7,Tajikistan,2016,8734951
8,Turkmenistan,2015,5565284
9,Turkmenistan,2016,5662544


Если мы отфильтруем этот `DataFrame`, то в индексе будут пропуски

In [41]:
pop_flat_df[pop_flat_df['year'] == 2015]

,country,year,pop
0,Afghanistan,2015,33736494
2,Kazakhstan,2015,17544126
4,Kyrgyzstan,2015,5956900
6,Tajikistan,2015,8548651
8,Turkmenistan,2015,5565284
10,Uzbekistan,2015,31298900


Чтобы создать заново проиндексировать этот `DataFrame` мы можем исопльзовать метод `reset_index`. Обратите внимание, что также передаем параметр `drop=True`, так как старый индекс нам уже не нужен

In [42]:
pop_flat_df[pop_flat_df['year'] == 2015].reset_index(drop=True)

,country,year,pop
0,Afghanistan,2015,33736494
1,Kazakhstan,2015,17544126
2,Kyrgyzstan,2015,5956900
3,Tajikistan,2015,8548651
4,Turkmenistan,2015,5565284
5,Uzbekistan,2015,31298900
